In [1]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy
from pennylane import BasisState, FermionicSingleExcitation, FermionicDoubleExcitation
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def inite(elec, orb):
    list1 = []
    # Single excitations
    list1.append([e for e in range(elec)])
    for x in range(elec):
        config = []
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    config.append(count + (1 if x % 2 else 0))
                else:
                    config.append(e)
            list1.append(config)
            config = []
            count += 2

    # Double excitations
    for x in range(elec):
        for y in range(x + 1, elec):
            for count1 in range(orb - elec, orb, 2):
                for count2 in range(orb - elec, orb, 2):
                    cont = 1 if (count1 != count2 or (x % 2) != (y % 2)) else 0
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        config = []
                        for e in range(elec):
                            if x == e:
                                config.append(count1 + (1 if x % 2 else 0))
                            elif y == e:
                                config.append(count2 + (1 if y % 2 else 0))
                            else:
                                config.append(e)
                        list1.append(config)
    return list1



In [2]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100, p_dephase=0.0,
             eps_z=0.0, eps_zz=0.0):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(
        symbols, geometry, basis="sto-3g", charge=charge, method="pyscf"
    )

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))

    # Device
   
    dev = qml.device("default.mixed", wires=qubits)
   

    # --- symmetry-preserving noise helpers (QSE-friendly) ---
    def _sym_preserving_noise_on(ws):
        ws = list(ws)

        # coherent single-qubit Z drift
        if eps_z != 0.0:
            for w in ws:
                qml.RZ(2.0 * eps_z, wires=w)

        # coherent ZZ residual coupling on all pairs in ws
        if eps_zz != 0.0 and len(ws) >= 2:
            for a in range(len(ws)):
                for b in range(a + 1, len(ws)):
                    qml.MultiRZ(2.0 * eps_zz, wires=[ws[a], ws[b]])

        # optional pure dephasing (population-preserving)
        if p_dephase != 0.0:
            for w in ws:
                qml.PhaseDamping(p_dephase, wires=w)



    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        BasisState(hf_state, wires=wires)

        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                FermionicDoubleExcitation(param_array[layer][len(s_wires) + i], wires1=w1, wires2=w2)
                # Noise on all wires touched by this excitation
            _sym_preserving_noise_on(list(w1) + list(w2))
                
            # Single excitations
            for j, s_w in enumerate(s_wires):
                FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            _sym_preserving_noise_on(list(s_w))
            
            #qml.BitFlip(0.01, wires=1)
        return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool


In [3]:
def qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params, shots=0,
           eps_z=0.0, eps_zz=0.0, p_dephase=0.0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(
        symbols, geometry, basis="sto-3g", method="pyscf",
        active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge
    )
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires = range(qubits)

    null_state = np.zeros(qubits, int)
    hf_state = qml.qchem.hf_state(active_electrons, qubits)
    list1 = inite(active_electrons, qubits)
    values = []

    if shots == 0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits, shots=shots)

    # --- symmetry-preserving noise helpers (QSE-friendly) ---
    def _sym_preserving_noise_on(ws):
        """Apply coherent Z / ZZ drift (and optional dephasing) on a set/list of wires."""
        ws = list(ws)

        # coherent single-qubit Z drift
        if eps_z != 0.0:
            for w in ws:
                qml.RZ(2.0 * eps_z, wires=w)

        # coherent ZZ residual coupling on all pairs in ws
        if eps_zz != 0.0 and len(ws) >= 2:
            for a in range(len(ws)):
                for b in range(a + 1, len(ws)):
                    qml.MultiRZ(2.0 * eps_zz, wires=[ws[a], ws[b]])

        # optional pure dephasing (population-preserving)
        if p_dephase != 0.0:
            for w in ws:
                qml.PhaseDamping(p_dephase, wires=w)

    # circuit for diagonal part
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)

        for w in occ:
            qml.PauliX(wires=w)
            #_sym_preserving_noise_on([w])

        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
            # noise on all qubits involved in this double excitation
        _sym_preserving_noise_on(list(w1) + list(w2))

        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        _sym_preserving_noise_on(list(s_w))

        return qml.expval(H)

    # circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)

        for w in occ1:
            qml.PauliX(wires=w)
            #_sym_preserving_noise_on([w])

        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                    #_sym_preserving_noise_on([v])
                else:
                    qml.CNOT(wires=[first, v])
                    #_sym_preserving_noise_on([first, v])

        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                    #_sym_preserving_noise_on([v])
                else:
                    qml.CNOT(wires=[first, v])
                    #_sym_preserving_noise_on([first, v])

        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        _sym_preserving_noise_on(list(w1) + list(w2))

        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        _sym_preserving_noise_on(list(s_w))

        return qml.expval(H)

    # final M matrix
    M = np.zeros((len(list1), len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i, i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)

    for i in range(len(list1)):
        for j in range(len(list1)):
            if i != j:
                Mtmp = circuit_od(params, list1[i], list1[j], wires, s_wires, d_wires, null_state)
                M[i, j] = Mtmp - M[i, i] / 2.0 - M[j, j] / 2.0

    eig, evec = np.linalg.eig(M)
    values.append(np.sort(eig))
    return values


In [4]:
r = 1.88973
#symbols = ['H', 'H', 'H', 'H']
symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 0.735*r]
    #[0.0, 1.5*r, 0.0],
    #[0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 2
charge = 0

active_electrons = 2
active_orbitals = 2

params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=100, p_dephase=0.05,
             eps_z=0.00, eps_zz=0.00)

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params, shots=0,
           eps_z=0.00, eps_zz=0.00, p_dephase=0.05)
print('eigenvalues:', eig)

Ground state energy: -1.13123376 Ha
eigenvalues: [tensor([-1.13123376, -0.49488423, -0.19248806,  0.48898185], requires_grad=True)]


eigenvalues: [tensor([-1.13730604, -0.52461743, -0.16275486,  0.49505412]